# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



quick-starts-ws-135610
aml-quickstarts-135610
southcentralus
a0a76bad-11a1-4a2d-9887-97a29122c8ed


In [3]:
# choose a name for experiment
experiment_name = 'capstone-Automl'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-Automl,quick-starts-ws-135610,Link to Azure Machine Learning studio,Link to Documentation


# Creat Compute Cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your GPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.get_status()

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


Data

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory


In [6]:
datastore= ws.get_default_datastore()

In [7]:
datastore.upload(src_dir= './', target_path= './')

Uploading an estimated of 3 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 3
Uploading ./heart_failure_clinical_records_dataset.csv
Uploaded ./heart_failure_clinical_records_dataset.csv, 2 files out of an estimated total of 3
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [8]:
tds= Dataset.Tabular.from_delimited_files(datastore.path('./heart_failure_clinical_records_dataset.csv'))

In [9]:
tds.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=tds,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion(show_output=True)




Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missin

{'runId': 'AutoML_f2ba87a0-9fdf-4292-b8eb-4f82d234cff9',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-21T16:25:00.317482Z',
 'endTimeUtc': '2021-01-21T16:47:12.994306Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-Automl","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-135610","workspace_name":"quick-starts-ws-135610","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featuri

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()


Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

In [14]:
print(best_run)
print(fitted_model)

NameError: name 'best_run' is not defined

In [16]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [20]:
description = 'HearFailure AutoML Model'
model = remote_run.register_model(description = description)

print(remote_run.model_id)

AutoMLf2ba87a0936


In [21]:
print(model.name, model.id, model.version, sep ='\t')

AutoMLf2ba87a0936	AutoMLf2ba87a0936:2	2


In [25]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.get(ws, "AzureML-AutoML-GPU")
env.inferencing_stack_version='latest'

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [28]:
from azureml.core.webservice import AciWebservice 
from azureml.core import Model

dep_cfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1) 

webservice = Model.deploy(ws, "capstone-automl", [model], inference_config, dep_cfg)

webservice.wait_for_deployment(show_output = True) 
print(webservice.state)  
print(webservice.scoring_uri) 
print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 246ce654-1e4d-4cc0-9823-d82698f3bc2b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'null' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'null' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 246ce654-1e4d-4cc0-9823-d82698f3bc2b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Error in entry script, NameError: name 'null' is not defined, please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, NameError: name 'null' is not defined, please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 246ce654-1e4d-4cc0-9823-d82698f3bc2b\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeploymentFailed\",\n  \"message\": \

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
output = webservice.run('./data.json')
print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
webservice.delete()